Neural Network is an algorithm that learns to identify patterns in the data

Backpropagation is a technique to train Neural Networks to update weights by minimising error value through Gradient Descent

Deep Learning = Many Layer Neural Net + Large amount of Data + Heavy Computation

In [1]:
import numpy as np
import pandas as pd
from basic_neural_net import NeuralNetwork
from sklearn.preprocessing import MinMaxScaler

In [2]:
#read data
train = pd.read_csv('titanic/train.csv',header=0)
test = pd.read_csv('titanic/test.csv',header=0)
minmax = MinMaxScaler()

In [3]:
#train.head()

#test.head()

#train.shape

#test.shape

#basic preprocessing

#sum(test.Sex.isnull())   #number of blank values in Sex

#sum(test.Embarked.isnull())  #number of blank values in Embarked   #remove 2 rows

#sum(test.Age.isnull())  #impute

#sum(test.Pclass.isnull())

#sum(test.SibSp.isnull())

#sum(test.Parch.isnull())

#sum(test.Cabin.isnull())  #delete this column

#sum(test.Fare.isnull())

#np.mean(test.Fare[test.Fare.notnull()].values)

#delete 'PassengerId','Name','Cabin' and 'Ticket'
#delete 2 blank 'Embarked' rows
#One hot 'Sex' and 'Embarked'
#delete 'Cabin'
#impute 177 rows of Age with some values or remove the column #say for now we impute it with the median of the values

In [4]:
def age_imputation(data,a,p,m):
    data1 = data[data.Age>a]
    data1 = data1[data1.Pclass==p]
    data1 = data1[data1.male==m]
    return np.mean(data1.Age.values)

def preprocessing(data):
    data.drop_duplicates(inplace=True)
    data.drop(['Ticket','Cabin'],axis=1,inplace=True)
    data = data[data.Embarked.notnull()]
    male = []
    female = []
    for i in data.Sex.values:
        if i == 'female':
            male.append(0)
            female.append(1)
        else:
            male.append(1)
            female.append(0)
    data['female'] = female
    data['male'] = male
    embarked_C = []
    embarked_Q = []
    embarked_S = []
    for i in data.Embarked.values:
        if i == 'C':
            embarked_C.append(1)
            embarked_Q.append(0)
            embarked_S.append(0)
        elif i == 'Q':
            embarked_C.append(0)
            embarked_Q.append(1)
            embarked_S.append(0)
        else:
            embarked_C.append(0)
            embarked_Q.append(0)
            embarked_S.append(1)
    data['embarked_c'] = embarked_C
    data['embarked_q'] = embarked_Q
    data['embarked_s'] = embarked_S
    data.Age.fillna(-1.0,inplace=True)
    data.Fare.fillna(np.mean(data.Fare[data.Fare.notnull()].values))
    age = []
    for i in range(data.shape[0]):
        if data.Age.values[i] != -1:
            age.append(data.Age.values[i])
        elif data.Age.values[i] == -1 and data.Pclass.values[i] == 1:
            if data.male.values[i] == 0:
                age.append(age_imputation(data,0,1,0))
            else:
                age.append(age_imputation(data,0,1,1))
        elif data.Age.values[i] == -1 and data.Pclass.values[i] == 2:
            if data.male.values[i] == 0:
                age.append(age_imputation(data,0,2,0))
            else:
                age.append(age_imputation(data,0,2,1))       
        elif data.Age.values[i] == -1 and data.Pclass.values[i] == 3:
            if data.male.values[i] == 0:
                age.append(age_imputation(data,0,3,0))
            else:
                age.append(age_imputation(data,0,3,1))   
    pclass_1 = []
    pclass_2 = []
    pclass_3 = []
    for i in data.Pclass.values:
        if i == 1:
            pclass_1.append(1)
            pclass_2.append(0)
            pclass_3.append(0)
        elif i == 2:
            pclass_1.append(0)
            pclass_2.append(1)
            pclass_3.append(0)
        else:
            pclass_1.append(0)
            pclass_2.append(0)
            pclass_3.append(1)
    data['pclass_1'] = pclass_1
    data['pclass_2'] = pclass_2
    data['pclass_3'] = pclass_3
    data.drop(['Pclass','Sex','Embarked'],axis=1,inplace=True)
    data['Age'] = age
    data['family_size'] = data.SibSp + data.Parch + 1
    data.Fare.fillna(-1.0,inplace=True)
    fare = []
    data_pclass_1 = data[data.pclass_1==1]
    data_pclass_2 = data[data.pclass_2==1]
    data_pclass_3 = data[data.pclass_3==1]
    fare_class_1 = data_pclass_1.Fare.values
    fare_class_2 = data_pclass_2.Fare.values
    fare_class_3 = data_pclass_3.Fare.values
    fare_mean_class1 = np.mean(fare_class_1)
    fare_mean_class2 = np.mean(fare_class_2)
    fare_mean_class3 = np.mean(fare_class_3)
    for i in range(data.shape[0]):
        if data.Fare.values[i] == -1 and data.pclass_1.values[i] == 1:
            fare.append(fare_mean_class1)
        elif data.Fare.values[i] == -1 and data.pclass_2.values[i] == 1:
            fare.append(fare_mean_class2)
        elif data.Fare.values[i] == -1 and data.pclass_3.values[i] == 1:
            fare.append(fare_mean_class3)
        else:
            fare.append(data.Fare.values[i])
    data.Fare = fare
    data.Fare = minmax.fit_transform(data.Fare)
    data.Age = minmax.fit_transform(data.Age)
    title = []
    for b in data.Name.values:
        title.append(b[(b.find(', ')+2):b.find('.')])
    data.Name = title
    mr = []
    mrs = []
    miss = []
    master = []
    mean_miss = np.mean(data[data.Name == 'Miss'].Age.values)
    mean_master = np.mean(data[data.Name == 'Master'].Age.values)
    for i in range(data.shape[0]):
        if title[i] in ['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Major','Mlle','Mme']:
            if data.male.values[i] == 0 and data.Age.values[i] <= mean_miss:
                miss.append(1)
                mr.append(0)
                mrs.append(0)
                master.append(0)
            elif data.male.values[i] == 0 and data.Age.values[i] > mean_miss:
                miss.append(0)
                mr.append(0)
                mrs.append(1)
                master.append(0)
            elif data.male.values[i] == 1 and data.Age.values[i] > mean_master:
                miss.append(0)
                mr.append(1)
                mrs.append(0)
                master.append(0)
            elif data.male.values[i] == 1 and data.Age.values[i] <= mean_master:
                miss.append(0)
                mr.append(0)
                mrs.append(0)
                master.append(1)
        elif title[i] in ['Lady','Mrs','the Countess']:
                miss.append(0)
                mr.append(0)
                mrs.append(1)
                master.append(0)
        elif title[i] == 'Master':
                miss.append(0)
                mr.append(0)
                mrs.append(0)
                master.append(1)
        elif title[i] in ['Miss','Ms']:
                miss.append(1)
                mr.append(0)
                mrs.append(0)
                master.append(0)
        else:
                miss.append(0)
                mr.append(1)
                mrs.append(0)
                master.append(0)
    data['miss'] = miss
    data['mr'] = mr
    data['mrs'] = mrs
    data['master'] = master
    data.drop('Name',axis=1,inplace=True)
    return data

In [5]:
#train_processed = preprocessing(train)
#test_processed = preprocessing(test)
pid_train = train.PassengerId.values
pid_test = test.PassengerId.values
total_Data = pd.DataFrame()
total_Data = total_Data.append(train)
total_Data = total_Data.append(test)
total_Data_processed = preprocessing(total_Data)
total_Data_processed = total_Data_processed[['Survived','Age', 'Fare', 'Parch', 'PassengerId', 'SibSp',
       'female', 'male', 'embarked_c', 'embarked_q', 'embarked_s',
       'pclass_1', 'pclass_2', 'pclass_3', 'family_size', 'miss', 'mr',
       'mrs', 'master']]
train = total_Data_processed[total_Data_processed.PassengerId.isin(pid_train)]
train.drop('PassengerId',axis=1,inplace=True)
test = total_Data_processed[total_Data_processed.PassengerId.isin(pid_test)]
test.drop('PassengerId',axis=1,inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [6]:
#np.random.seed(1)

In [7]:
val_id = np.random.choice(range(train.shape[0]), 150, replace=False)

In [8]:
train_id = list(set(range(train.shape[0])) - set(list(val_id)))

In [9]:
validation_set = train.iloc[val_id]

In [10]:
train_processed_final = train.iloc[train_id]

In [11]:
x_train = train_processed_final.iloc[:,1:].values
x_valid = validation_set.iloc[:,1:].values
x_test = test.values

In [12]:
y_train = train_processed_final.iloc[:,0]
y_valid = validation_set.iloc[:,0]

In [13]:
nn = NeuralNetwork()

In [14]:
iters = 10000
learning_rate = 0.000001
nn.train(x=x_train,y=y_train,iterations=iters,learning_rate=learning_rate,batch_size=200)

In [15]:
y_pred = nn.predict(x_train)

In [16]:
nn.accuracy(y_pred,y_train)

38.565629228687413

In [17]:
valid_pred = nn.predict(x_valid)
nn.accuracy(valid_pred,y_valid)

36.666666666666664

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rfc = RandomForestClassifier(criterion='entropy',min_samples_leaf=1,min_samples_split=2)

In [20]:
rfc.fit(X=x_train,y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
rfc.score(X=x_train,y=y_train)

0.96752368064952643

In [22]:
rfc.score(X=x_valid,y=y_valid)

0.78000000000000003

In [23]:
import tensorflow as tf

In [24]:
xtf = tf.placeholder(tf.float32,[None,x_train.shape[1]])
ytf = tf.placeholder(tf.float32,[None,1])

In [25]:
w = tf.Variable(tf.zeros([x_train.shape[1],1]))
b = tf.Variable(tf.zeros([1]))

In [26]:
logit = tf.add(tf.matmul(xtf,w),b)
activation = tf.nn.softmax(logit)

In [27]:
cost = tf.reduce_mean(tf.reduce_sum(tf.square(ytf-activation),reduction_indices=[1]))

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.000001).minimize(cost)

In [29]:
def accuracy(ax,ay):
    ypred = sess.run(activation,feed_dict={xtf:ax})
    acty = sess.run(ytf,feed_dict={ytf:ay})
    ypredn = []
    for i in ypred:
        if i<0.5:
            ypredn.append(1)
        else:
            ypredn.append(0)
    ypredn = np.asarray(ypredn)
    return np.mean(np.equal(ypredn,acty))*100

In [30]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [31]:
y_train = y_train.reshape([y_train.shape[0],1])
y_valid = y_valid.reshape([y_valid.shape[0],1])
for i in range(10000):
    batch_ids = np.random.choice(range(train_processed_final.shape[0]), 200, replace=False)
    train_batch_final = train_processed_final.iloc[batch_ids]
    x_batch = train_batch_final.iloc[:,1:].values
    y_batch = train_batch_final.iloc[:,0]
    y_batch = y_batch.reshape([y_batch.shape[0],1])
    sess.run(optimizer,feed_dict={xtf:x_batch,ytf:y_batch})
    if i%1000 == 0:
        c = sess.run(cost,feed_dict={xtf:x_batch,ytf:y_batch})
        at = accuracy(x_train,y_train)
        av = accuracy(x_valid,y_valid)
        print "cost : %0.3f, accuracy train : %0.3f, accuracy valid : %0.3f"%(c,at,av)

cost : 0.635, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.665, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.620, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.610, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.605, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.625, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.640, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.625, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.620, accuracy train : 61.434, accuracy valid : 63.333
cost : 0.595, accuracy train : 61.434, accuracy valid : 63.333
